### Notebook for concept detection in neural network

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.concept_win_on_pass

CONCEPT_NAME = concepts.concept_win_on_pass.__name__


2023-10-26 14:02:06.322091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [4]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2
    alpha = 0.2 # 0 = argmax, 1 = distribution

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state, alpha=alpha)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 2500 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases: 2519it [09:12,  4.95it/s]                          

In [5]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 4, 5, 5)
Negative cases:  (2500, 4, 5, 5)


In [6]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [9]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  29
Duplicate count in negative:  323


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [7]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 4000


In [8]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.9297 - val_loss: 0.8890
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8452 - val_loss: 0.8163
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7773 - val_loss: 0.7564
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7212 - val_loss: 0.7066
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6737 - val_loss: 0.6635
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6327 - val_loss: 0.6262
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5967 - val_loss: 0.5928
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5645 - val_loss: 0.5628
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5355 - val_loss: 0.5359
Epoch 10/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5093 - val_loss: 0.5112

In [10]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 276.75it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.8981 - val_loss: 0.8445
Epoch 2/50
125/125 [==============================] - 0s 948us/step - loss: 0.7957 - val_loss: 0.7596
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7185 - val_loss: 0.6929
Epoch 4/50
125/125 [==============================] - 0s 957us/step - loss: 0.6570 - val_loss: 0.6387
Epoch 5/50
125/125 [==============================] - 0s 947us/step - loss: 0.6070 - val_loss: 0.5945
Epoch 6/50
125/125 [==============================] - 0s 925us/step - loss: 0.5664 - val_loss: 0.5590
Epoch 7/50
125/125 [==============================] - 0s 955us/step - loss: 0.5340 - val_loss: 0.5311
Epoch 8/50
125/125 [==============================] - 0s 940us/step - loss: 0.5096 - val_loss: 0.5107
Epoch 9/50
125/125 [==============================] - 0s 960us/step - loss: 0.4918 - val_loss: 0.4960
Epoch 10/50
125/125 [==============================]

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 249.87it/s]

Performing regression for layer 0
Epoch 1/50


125/125 [==============================] - 0s 2ms/step - loss: 0.9943 - val_loss: 0.8637
Epoch 2/50
125/125 [==============================] - 0s 921us/step - loss: 0.7865 - val_loss: 0.7162
Epoch 3/50
125/125 [==============================] - 0s 929us/step - loss: 0.6615 - val_loss: 0.6157
Epoch 4/50
125/125 [==============================] - 0s 923us/step - loss: 0.5740 - val_loss: 0.5443
Epoch 5/50
125/125 [==============================] - 0s 921us/step - loss: 0.5108 - val_loss: 0.4913
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4631 - val_loss: 0.4509
Epoch 7/50
125/125 [==============================] - 0s 927us/step - loss: 0.4256 - val_loss: 0.4187
Epoch 8/50
125/125 [==============================] - 0s 914us/step - loss: 0.3952 - val_loss: 0.3919
Epoch 9/50
125/125 [==============================] - 0s 920us/step - loss: 0.3698 - val_loss: 0.3692
Epoch 10/50
125/125 [==============================] - 0s 918us/step - loss: 0.3481 - val_loss: 0

...

Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9522 - val_loss: 0.8170
Epoch 2/50
125/125 [==============================] - 0s 948us/step - loss: 0.7354 - val_loss: 0.6786
Epoch 3/50
125/125 [==============================] - 0s 960us/step - loss: 0.6251 - val_loss: 0.5934
Epoch 4/50
125/125 [==============================] - 0s 962us/step - loss: 0.5523 - val_loss: 0.5327
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4989 - val_loss: 0.4864
Epoch 6/50
125/125 [==============================] - 0s 944us/step - loss: 0.4573 - val_loss: 0.4498
Epoch 7/50
125/125 [==============================] - 0s 943us/step - loss: 0.4240 - val_loss: 0.4200
Epoch 8/50
125/125 [==============================] - 0s 958us/step - loss: 0.3966 - val_loss: 0.3950
Epoch 9/50
125/125 [==============================] - 0s 939us/step - loss: 0.3730 - val_loss: 0.3733
Epoch 10/50
125/125 [==============================]

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 288.29it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.7941 - val_loss: 0.7255
Epoch 2/50
125/125 [==============================] - 0s 980us/step - loss: 0.6648 - val_loss: 0.6242
Epoch 3/50
125/125 [==============================] - 0s 923us/step - loss: 0.5788 - val_loss: 0.5525
Epoch 4/50
125/125 [==============================] - 0s 965us/step - loss: 0.5162 - val_loss: 0.4983
Epoch 5/50
125/125 [==============================] - 0s 944us/step - loss: 0.4681 - val_loss: 0.4556
Epoch 6/50
125/125 [==============================] - 0s 940us/step - loss: 0.4299 - val_loss: 0.4214
Epoch 7/50
125/125 [==============================] - 0s 980us/step - loss: 0.3988 - val_loss: 0.3935
Epoch 8/50
125/125 [==============================] - 0s 972us/step - loss: 0.3732 - val_loss: 0.3702
Epoch 9/50
125/125 [==============================] - 0s 990us/step - loss: 0.3519 - val_loss: 0.3510
Epoch 10/50
125/125 [=============================

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 292.93it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.8842 - val_loss: 0.7699
Epoch 2/50
125/125 [==============================] - 0s 999us/step - loss: 0.7038 - val_loss: 0.6385
Epoch 3/50
125/125 [==============================] - 0s 957us/step - loss: 0.5975 - val_loss: 0.5527
Epoch 4/50
125/125 [==============================] - 0s 975us/step - loss: 0.5248 - val_loss: 0.4923
Epoch 5/50
125/125 [==============================] - 0s 958us/step - loss: 0.4712 - val_loss: 0.4458
Epoch 6/50
125/125 [==============================] - 0s 938us/step - loss: 0.4304 - val_loss: 0.4098
Epoch 7/50
125/125 [==============================] - 0s 939us/step - loss: 0.3973 - val_loss: 0.3798
Epoch 8/50
125/125 [==============================] - 0s 938us/step - loss: 0.3700 - val_loss: 0.3549
Epoch 9/50
125/125 [==============================] - 0s 926us/step - loss: 0.3465 - val_loss: 0.3330
Epoch 10/50
125/125 [=============================